In [1]:
#Imports

import praw
from praw.models import MoreComments
import gpt_2_simple as gpt2
import time
import datetime
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess)

C:\Users\johng\anaconda3\envs\pyvizenv\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\johng\anaconda3\envs\pyvizenv\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\johng\anaconda3\envs\pyvizenv\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


Loading checkpoint checkpoint\run1\model-1000
INFO:tensorflow:Restoring parameters from checkpoint\run1\model-1000


ResourceExhaustedError: Graph execution error:

SameWorkerRecvDone unable to allocate output tensor. Key: /job:localhost/replica:0/task:0/device:CPU:0;0000000000000001;/job:localhost/replica:0/task:0/device:GPU:0;edge_300_save/RestoreV2;0:0
	 [[{{node save/RestoreV2/_295}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

In [ ]:
#load reddit credentials from env file
import os 
from dotenv import load_dotenv
load_dotenv()

d_id = os.getenv("RID")
c_secret = os.getenv("RC_SECRET")
REDDIT_USERNAME = os.getenv("REDDIT_USERNAME")
REDDIT_PASS = os.getenv("REDDIT_PASS")
user_agent = os.getenv("user_agent")


In [ ]:
#Initial reddit object/session and set subreddit
reddit = praw.Reddit(username=REDDIT_USERNAME,password=REDDIT_PASS,client_id=d_id,client_secret=c_secret,user_agent=user_agent)    
subreddit = reddit.subreddit("darksouls3")


In [ ]:
#Test login session, should print bot's reddit user name
print(reddit.user.me())

In [ ]:
#Scrap subreddits hot posts
for submission in reddit.subreddit('darksouls3').hot(limit=4):
    print(submission.title)
    sub_id = submission.id
    post_id = sub_id

In [ ]:
#Scrap comments
comments = {"submid":[], "text":[]}
submission = reddit.submission(id = sub_id)
submission.comment_sort = "new"
submission.comment_limit = 1
for comment in submission.comments:
    if isinstance(comment, MoreComments):
        continue
    comments["submid"].append(comment.link_id)
    comments["text"].append(comment.body)

In [ ]:
#Single out comment text we want to use for the input
input_text = str(comments["text"])
input_text

In [ ]:
#Generate text using model based on input
output = gpt2.generate_to_file(sess,prefix=input_text,temperature=0.8,length=120)

In [ ]:
#Write to file text because I couldn't fucking figure out the decoder bit
text_file = open("gpt_2_gen_texts.txt", "r")
text = text_file.read()
text_file.close()
print(text)

In [ ]:
#Read from file to get response in string format, because decoders, how do they work? Fuck it, it works
output_text = text.replace(input_text,'')

In [ ]:
disclaimer = '.                                                      ***I am a bot built using GPT2 and trained on the Dark Soul 3 NPC dialogue text, still in testing***'
listOfStrings = [output_text, disclaimer]
finalString = "".join(listOfStrings)

In [ ]:
submission.comments.reply(finalString)